# Phase 4: Output Formatting for Claude Code

This notebook handles:
1. Loading the generated checkpoints from Phase 3
2. Formatting checkpoints into XML format for Claude Code
3. Generating context priming prompts
4. Saving the final formatted outputs

In [1]:
# Import shared utilities
import sys
sys.path.append('.')

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
import os

from utils.interim_data_management import load_interim_data, save_interim_data
from utils.config import get_config

## Load Phase 3 Results

In [3]:
# Get config
config = get_config()

# Load previous phase data
raw_inputs = load_interim_data(config.get("INTERIM_DATA_DIR"), "raw_inputs", "phase1")
generated_checkpoints_data = load_interim_data(config.get("INTERIM_DATA_DIR"), "generated_checkpoints", "phase3")
phase3_summary = load_interim_data(config.get("INTERIM_DATA_DIR"), "phase3_summary", "phase3")

# Extract key variables
INPUT_NAME = raw_inputs["input_name"]
file_tree = raw_inputs["file_tree"]

print(f"Loaded {len(generated_checkpoints_data)} checkpoints from Phase 3")
print(f"Input name: {INPUT_NAME}")

Loaded raw_inputs from interim_data/phase1/raw_inputs.json
Loaded generated_checkpoints from interim_data/phase3/generated_checkpoints.json
Loaded phase3_summary from interim_data/phase3/phase3_summary.json
Loaded 7 checkpoints from Phase 3
Input name: context_manager_update_frontend


## Reconstruct Checkpoint Models

In [4]:
# Recreate the model definitions (same as Phase 3)
class LowLevelTask(BaseModel):
    task_id: str
    file_path: str
    description: str
    action_details: str
    complexity: int
    depends_on: Optional[List[str]] = Field(default_factory=list)

class TaskTranche(BaseModel):
    tranche_id: str
    goal: str
    low_level_tasks: List[LowLevelTask]

class RequestCheckpointExecutionSpecification(BaseModel):
    implementation_notes_specific_to_checkpoint: Optional[List[str]] = Field(default_factory=list)
    task_tranches: List[TaskTranche]

class RequestCheckpointFileContextItem(BaseModel):
    file_path: str
    status_description: str

class RequestCheckpointSpecificFileContext(BaseModel):
    beginning_files_for_request_checkpoint: List[RequestCheckpointFileContextItem]
    ending_files_for_request_checkpoint: List[RequestCheckpointFileContextItem]

class RequestCheckpoint(BaseModel):
    order: int
    title: str
    goal_for_request_checkpoint: str
    prerequisites: Optional[List[str]] = Field(default_factory=list)
    expected_outcome_or_deliverables: Optional[str] = None
    request_checkpoint_specific_file_context: RequestCheckpointSpecificFileContext
    execution_spec: RequestCheckpointExecutionSpecification

# Convert loaded data back to Pydantic models
generated_checkpoints = [
    RequestCheckpoint(**checkpoint_data) for checkpoint_data in generated_checkpoints_data
]

## Define Formatting Functions

In [5]:
def format_checkpoint_to_claude_code_prompts(checkpoint: RequestCheckpoint) -> str:
    """
    Formats a RequestCheckpoint object into an XML string for Claude Code.
    """
    request_string = "<request>\n"
    request_string += "\t<user_input>\n"
    request_string += f"\t\tIngest the information from this file, implement the <commands> within the <task_tranches> of the <execution_spec>\n"
    request_string += "\t\tGenerate the code that will satisfy the specified <goal> and <expected_outcome> of the checkpoint.\n"
    request_string += "\t\tAdhere to the <implementation_notes> defined in the <execution_spec>.\n"
    request_string += "\t\tTasks within a tranche should be executed in parallel, as they are non blocking within a tranche."
    request_string += "\t\tIf you need more context, run `eza . --tree --git-ignore` to get the file tree.  Parallel read the files you find relevant that you have not already read."
    request_string += "\t</user_input>\n"
    request_string += f'\t<plan title="{checkpoint.title}">\n'
    request_string += f"\t\t<goal>{checkpoint.goal_for_request_checkpoint}</goal>\n"
    request_string += f"\t\t<expected_outcome>{checkpoint.expected_outcome_or_deliverables}</expected_outcome>\n"
    request_string += "\t\t<execution_spec>\n"
    request_string += "\t\t\t<starting_context>\n"
    for file in checkpoint.request_checkpoint_specific_file_context.beginning_files_for_request_checkpoint:
        request_string += f"\t\t\t\t<file path=\"{file.file_path}\">\n"
        request_string += f"\t\t\t\t\t<status>{file.status_description}</status>\n"
        request_string += f"\t\t\t\t</file>\n"
    request_string += "\t\t\t</starting_context>\n"
    request_string += "\t\t\t<ending_context>\n"
    for file in checkpoint.request_checkpoint_specific_file_context.ending_files_for_request_checkpoint:
        request_string += f"\t\t\t\t<file path=\"{file.file_path}\">\n"
        request_string += f"\t\t\t\t\t<status>{file.status_description}</status>\n"
        request_string += f"\t\t\t\t</file>\n"
    request_string += "\t\t\t</ending_context>\n"
    request_string += f"\t\t\t<implementation_notes>\n"
    for note in checkpoint.execution_spec.implementation_notes_specific_to_checkpoint:
        request_string += f"\t\t\t\t<note>{note}</note>\n"
    request_string += f"\t\t\t</implementation_notes>\n"
    request_string += f"\t\t\t<task_tranches>\n"
    for tranche in checkpoint.execution_spec.task_tranches:
        request_string += f"\t\t\t\t<task_tranche id=\"{tranche.tranche_id}\">\n"
        request_string += f"\t\t\t\t\t<goal>{tranche.goal}</goal>\n"
        request_string += f"\t\t\t\t\t<low_level_tasks>\n"
        for task in tranche.low_level_tasks:
            request_string += f"\t\t\t\t\t<low_level_task id=\"{task.task_id}\">\n"
            request_string += f"\t\t\t\t\t\t<file_path>{task.file_path}</file_path>\n"
            request_string += f"\t\t\t\t\t\t<description>{task.description}</description>\n"
            request_string += f"\t\t\t\t\t\t<command>{task.action_details}</command>\n"
            request_string += "===\n"
            request_string += "```aider\n"
            request_string += f"{task.action_details}\n"
            request_string += "```\n"
            request_string += "===\n"
            request_string += f"\t\t\t\t\t</low_level_task>\n"
        request_string += f"\t\t\t\t</low_level_tasks>\n"
    request_string += f"\t\t\t</task_tranches>\n"
    request_string += f"\t\t</execution_spec>\n"
    request_string += "\t</plan>\n"
    request_string += "</request>\n"
    return request_string

In [6]:
def make_context_prime_prompt(
    file_tree_content: str,
    checkpoint: RequestCheckpoint
) -> str:
    """
    Generate a context priming prompt for a checkpoint.
    """
    request_string = '<request name="context_priming">\n'
    request_string += f'\t<user_input>\n'
    request_string += f"\t\tFollow the instructions precisely. If it wasn't specified, don't do it.\n"
    request_string += f"\t</user_input>\n"
    request_string += f"\t<instructions>\n"
    request_string += f"\t\t<instruction content='Parallel Read the following files'>\n"
    for file_info in checkpoint.request_checkpoint_specific_file_context.beginning_files_for_request_checkpoint:
        request_string += f'\t\t\t<file_path>"{file_info.file_path}"</file_path>\n'
    request_string += f'\t\t</instruction>\n'
    request_string += f"\t\t<instruction content='Ingest the following file tree. Do not read any files, just get an understanding of the file structure'>\n"
    request_string += f"\t\t\t{file_tree_content}\n"
    request_string += f"\t\t</instruction>\n"
    request_string += f'\t</instructions>\n'
    request_string += f'</request>\n'
    
    return request_string

## Generate and Save Formatted Outputs

In [7]:
# Create output directory
output_dir = os.path.join(config.get("OUTPUT_DIR"), "checkpoints_xml_claude_code")
print(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Also create a directory for context priming prompts
context_prime_dir = os.path.join(config.get("OUTPUT_DIR"), "context_priming_prompts")
print(context_prime_dir)
os.makedirs(context_prime_dir, exist_ok=True)

print(f"Formatting {len(generated_checkpoints)} checkpoints for Claude Code...\n")

outputs/checkpoints_xml_claude_code
outputs/context_priming_prompts
Formatting 7 checkpoints for Claude Code...



In [8]:
# Process each checkpoint
formatted_outputs = []

for i, checkpoint in enumerate(generated_checkpoints):
    # Format the checkpoint to XML
    xml_str = format_checkpoint_to_claude_code_prompts(checkpoint)
    
    # Create filename (sanitize the title for filesystem)
    safe_title = checkpoint.title.replace("/", "_").replace(":", "_").replace("?", "_")
    filename = f"checkpoint_{i}_{safe_title}.xml"
    file_path = os.path.join(output_dir, filename)
    
    # Save the XML file
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(xml_str)
    
    print(f"✓ Saved checkpoint {i+1}: {filename}")
    
    # Generate and save context priming prompt
    context_prime = make_context_prime_prompt(file_tree, checkpoint)
    context_filename = f"context_prime_{i}_{safe_title}.xml"
    context_path = os.path.join(context_prime_dir, context_filename)
    
    with open(context_path, "w", encoding="utf-8") as f:
        f.write(context_prime)
    
    # Store formatted output info
    formatted_outputs.append({
        "checkpoint_order": checkpoint.order,
        "checkpoint_title": checkpoint.title,
        "xml_file": file_path,
        "context_prime_file": context_path,
        "tranches_count": len(checkpoint.execution_spec.task_tranches),
        "total_tasks": sum(len(tranche.low_level_tasks) for tranche in checkpoint.execution_spec.task_tranches)
    })

print(f"\nGenerated {len(formatted_outputs)} XML files for Claude Code")
print(f"Output directory: {output_dir}")
print(f"Context priming directory: {context_prime_dir}")

✓ Saved checkpoint 1: checkpoint_0_Implement New Server Actions for Context Bundles.xml
✓ Saved checkpoint 2: checkpoint_1_Refactor Legacy `resolveContextSelectionAction`.xml
✓ Saved checkpoint 3: checkpoint_2_Implement Frontend State Management for Active Context Bundle.xml
✓ Saved checkpoint 4: checkpoint_3_Integrate Context Bundle ID into Frontend Operations and State.xml
✓ Saved checkpoint 5: checkpoint_4_Implement Frontend Display of Context Bundle Snapshots on Cards.xml
✓ Saved checkpoint 6: checkpoint_5_Update Documentation to Reflect Server-Action Architecture.xml
✓ Saved checkpoint 7: checkpoint_6_End-to-End Testing and Quality Assurance.xml

Generated 7 XML files for Claude Code
Output directory: outputs/checkpoints_xml_claude_code
Context priming directory: outputs/context_priming_prompts


## Generate Summary Report

In [9]:
# Create a markdown summary report
summary_report = f"# Spec Generation Summary for {INPUT_NAME}\n\n"
summary_report += f"## Overview\n\n"
summary_report += f"- **Total Checkpoints Generated**: {len(generated_checkpoints)}\n"
summary_report += f"- **Total Task Tranches**: {sum(len(cp.execution_spec.task_tranches) for cp in generated_checkpoints)}\n"
summary_report += f"- **Total Low-Level Tasks**: {sum(sum(len(t.low_level_tasks) for t in cp.execution_spec.task_tranches) for cp in generated_checkpoints)}\n\n"

summary_report += "## Checkpoints\n\n"
for output in formatted_outputs:
    summary_report += f"### {output['checkpoint_order']}. {output['checkpoint_title']}\n"
    summary_report += f"- **Tranches**: {output['tranches_count']}\n"
    summary_report += f"- **Tasks**: {output['total_tasks']}\n"
    summary_report += f"- **Files**:\n"
    summary_report += f"  - XML: `{os.path.basename(output['xml_file'])}`\n"
    summary_report += f"  - Context Prime: `{os.path.basename(output['context_prime_file'])}`\n\n"

summary_report += "## Usage Instructions\n\n"
summary_report += "1. For each checkpoint, first run the context priming prompt to load relevant files\n"
summary_report += "2. Then run the main checkpoint XML file to execute the implementation\n"
summary_report += "3. Checkpoints should be executed in order, as later checkpoints may depend on earlier ones\n"

# Save the summary report
summary_path = os.path.join(config.get("OUTPUT_DIR"), f"summary_{INPUT_NAME}.md")
with open(summary_path, "w") as f:
    f.write(summary_report)

print(f"\nSaved summary report to: {summary_path}")


Saved summary report to: outputs/summary_context_manager_update_frontend.md


## Save Phase 4 Results

In [10]:
# Save Phase 4 summary
phase4_summary = {
    "input_name": INPUT_NAME,
    "checkpoints_formatted": len(formatted_outputs),
    "output_directory": output_dir,
    "context_prime_directory": context_prime_dir,
    "summary_report_path": summary_path,
    "formatted_outputs": formatted_outputs
}

save_interim_data(config.get("INTERIM_DATA_DIR"), phase4_summary, "phase4_summary", "phase4")

print("\nPhase 4 completed successfully!")
print(f"All outputs saved to: {config.get('OUTPUT_DIR')}")
print("\n✨ Spec generation pipeline complete! ✨")

Saved phase4_summary to interim_data/phase4/phase4_summary.json

Phase 4 completed successfully!
All outputs saved to: outputs

✨ Spec generation pipeline complete! ✨
